### Imports

In [66]:
import numpy as np
import pandas as pd
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
from pytensor.compile.builders import OpFromGraph
from time import perf_counter
from collections import defaultdict
import pymc_extras as pmx
from pymc_extras.statespace import structural as sts
import pytensor
from pytensor.graph.basic import explicit_graph_inputs
import numpy as np

### Generate a random dataset

In [67]:
mod = (
    sts.LevelTrendComponent(order=2, innovations_order=[0, 1], name='level') +
    sts.AutoregressiveComponent(order=1, name='ar') +
    sts.MeasurementError(name='obs_error')
).build(verbose = False)

param_values = {
    'initial_level': np.array([10, 0.1]),
    'sigma_level': np.array([1e-2]),
    'params_ar': np.array([0.95]),
    'sigma_ar': np.array(1e-2),
    'sigma_obs_error': np.array(1e-2),
}

data_fn = pmx.statespace.compile_statespace(mod, steps=100)
hidden_state_data, obs_data = data_fn(**param_values)

matrices = mod._unpack_statespace_with_placeholders()

matrix_fn = pytensor.function(list(explicit_graph_inputs(matrices)),
                                   matrices)
a0, P0, c, d, T, Z, R, H, Q = matrix_fn(**param_values, initial_state_cov=np.eye(mod.k_states))

### Symbolic variable

In [68]:
# Paramètres symboliques
A_sym = pt.matrix("A")              # (n, n)
H_sym = pt.matrix("H")              # (n, n)
Q_sym = pt.matrix("Q")              # (n, n)
R_sym = pt.matrix("R")              # (n, n)
T_sym = pt.matrix("T")              # (n, n)
Z_sym = pt.matrix("Z")              # (n, n)
y_sym = pt.matrix("y")              # (T, n) : observations

a0_sym = pt.vector("a0")            # (n,)        
P0_sym = pt.matrix("P0")            # (n, n)

data_sym = pt.matrix('data_sym')    # [T, obs_dim]

## Kalman filter with classic gradient

### The Loss

The Negative Log-Likelihood loss os given in the paper as the following expression :

$$
L_{NLL} = \sum l_{n|n} + l_{n|n-1}
$$

Where :

$$
\begin{align}
&l_{n|n} = 0 \\
&l_{n|n-1} = log det(F) + v_n^TFv_n
\end{align}
$$

In [69]:
def predict(a, P, T, Q):
    a_hat = T @ a                               # x_n|n-1
    P_hat = T @ P @ T.T + Q                     # P_n|n-1
    return a_hat, P_hat

def update(y, a, P, Z, H):
    v = y - Z.dot(a)                            # z_n
    PZT = P.dot(Z.T)                            

    F = Z.dot(PZT) + H                          # S_n
    F_inv = pt.linalg.inv(F)                    # S_n^(-1)
    K = PZT.dot(F_inv)                          # K_n

    I_KZ = pt.eye(a.shape[0]) - K.dot(Z)
    a_filtered = a + K.dot(v)                   # x_n|n
    P_filtered = I_KZ @ P                       # P_n|n

    inner_term = v.T @ F_inv @ v
    _, F_logdet = pt.linalg.slogdet(F)          # log det S_n
    ll = (F_logdet + inner_term).ravel()[0]     # Loss

    return [a_filtered, P_filtered, Z.dot(a), F, ll]

def kalman_step(y, a, P, T, Z, H, Q):
    a_filtered, P_filtered, obs_mu, obs_cov, ll = update(y=y, a=a, P=P, Z=Z, H=H)
    a_hat, P_hat = predict(a=a_filtered, P=P_filtered, T=T, Q=Q)
    return [a_filtered, a_hat, obs_mu, P_filtered, P_hat, obs_cov, ll]


outputs_info = [None, a0_sym, None, None, P0_sym, None, None]

results_seq, updates = pytensor.scan(
    kalman_step,
    sequences=[data_sym],
    outputs_info=outputs_info,
    non_sequences=[T_sym, Z_sym, H_sym, Q_sym],
    strict=False,
)

# --- Loss ---
a_upd_seq, a_pred_seq, y_hat_seq, P_upd_seq, P_pred_seq, obs_cov, ll_seq = results_seq
loss = pt.sum(ll_seq)

## Custom gradient

### Gradient with respect to **$a_{n-1|n-1}$**

From the article we have :

$$
\begin{align}
&\frac{dL}{da_{n-1|n-1}} = T_n^T \frac{dL}{da_{n|n-1}} 
+ \frac{dl_{n-1|n-1}}{da_{n-1|n-1}} \quad &\text{(equation 22)} \\
&\frac{dl_{n|n}}{da_{n|n}} = 0 \quad &\text{(equation 28)}
\end{align}
$$

Givent this two equations, we now have :
$$
\begin{align}
&\frac{dL}{da_{n-1|n-1}} = T_n^T \frac{dL}{da_{n|n-1}} 
\end{align}
$$

### Gradient with respect to **$P_{n-1|n-1}$**

From the article we have :

$$
\begin{align}
&\frac{dL}{dP_{n-1|n-1}} = T_n^T \frac{dL}{dP_{n|n-1}} T_n
+ \frac{dl_{n-1|n-1}}{dP_{n-1|n-1}} \quad &\text{(equation 23)} \\
&\frac{dl_{n|n}}{dP_{n|n}} = 0 \quad &\text{(equation 28)}
\end{align}
$$

Givent this two equations, we now have :
$$
\begin{align}
&\frac{dL}{dP_{n-1|n-1}} = T_n^T \frac{dL}{dP_{n|n-1}} T_n
\end{align}
$$


### Gradient with respect to **$a_{n|n-1}$**

From the article we have :

$$
\begin{align}
&\frac{dL}{da_{n|n-1}} = (I - K_n Z_n)^T \frac{dL}{da_{n|n}} + \frac{dl_{n|n-1}}{da_{n|n-1}} \quad &\text{(equation 20)} \\
&\frac{dl_{n|n-1}}{da_{n|n-1}} = -2 Z_n^{T}F_n^{-1} v_n \quad &\text{(equation 30)} \\
&\frac{dL}{da_{n|n}} = T_n^T \frac{dL}{da_{n+1|n}} \quad &\text{see gradient with respect to} \quad a_{n|n}
\end{align}
$$

Givent this two equations, we now have :
$$
\begin{align}
&\frac{dL}{da_{n|n-1}} = (I - K_n Z_n)^T T_n^T \frac{dL}{da_{n+1|n}} - 2 Z_n^{T}F^{-1} v_n
\end{align}
$$


In [70]:
def grad_a_hat(inp, out, out_grad):
    y, a, P, T, Z, H, Q = inp
    a_hat_grad, _, _ = out_grad

    v = y - Z.dot(a)                          

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H                          
    F_inv = pt.linalg.inv(F)
    
    K = PZT.dot(F_inv)                          
    I_KZ = pt.eye(a.shape[0]) - K.dot(Z)

    grad_a_pred = I_KZ.T @ T.T @ a_hat_grad - 2 * Z.T @ F_inv @ v

    return grad_a_pred

### Gradient with respect to **$P_{n|n-1}$**

From the article we have :

$$
\begin{align}
&\frac{dL}{dP_{n|n-1}} = (I - K_n Z_n)^T [
    \frac{dL}{dP_{n|n}}
    + \frac{1}{2} \frac{dL}{da_{n|n}} v_n^T H_n^-1 Z_n
    + \frac{1}{2} Z_n^T R_n^{-1} v_n (\frac{dL}{da_{n|n}})^T
    ](I - K_n Z_n) 
    + \frac{dl{n|n-1}}{dP_{n|n-1}}  \quad &\text{(equation 21)} \\
&\frac{dl_{n|n-1}}{dP_{n|n-1}} = Z_n^T F_n^{-1} Z_n - Z_n^T F_n^-1 v_n v_n^T F_n^{-1} Z_n \quad &\text{(equation 29)} \\
&\frac{dL}{da_{n|n}} = T_n^T \frac{dL}{da_{n+1|n}} \quad &\text{see gradient with respect to} \quad a_{n|n} \\
&\frac{dL}{dP_{n|n}} = T_n^T \frac{dL}{dP_{n+1|n}} T_n \quad &\text{see gradient with respect to} \quad P_{n|n}
\end{align}
$$

Givent this two equations, we now have :
$$
\begin{align}
&\frac{dL}{dP_{n|n-1}} = (I - K_n Z_n)^T [
    T_n^T \frac{dL}{dP_{n+1|n}} T_n
    + \frac{1}{2} T_n^T \frac{dL}{da_{n+1|n}} v_n^T H_n^{-1} Z_n
    + \frac{1}{2} Z_n^T H_n^{-1} v_n (T_n^T \frac{dL}{da_{n+1|n}})^T
    ](I - K_n Z_n) 
    + Z_n^T F_n^{-1} Z_n 
    - Z_n^T F_n^{-1} v_n v_n^T F_n^{-1} Z_n
\end{align}
$$

In [71]:
def grad_P_hat(inp, out, out_grad):
    y, a, P, T, Z, H, Q = inp
    a_hat_grad, P_hat_grad, ll_grad = out_grad

    v = y - Z.dot(a)
    v = v.dimshuffle(0, 'x')
    a_hat_grad = a_hat_grad.dimshuffle(0, 'x') 

    P_filtered_grad = T.T @ P_hat_grad @ T
    a_filtered_grad = T.T @ a_hat_grad                     

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H

    H_inv = pt.linalg.inv(H)                         
    F_inv = pt.linalg.inv(F)
    
    K = PZT.dot(F_inv)                          
    I_KZ = pt.eye(a.shape[0]) - K.dot(Z)

    grad_P_hat = I_KZ.T @ ( P_filtered_grad + 0.5 * a_filtered_grad @ v.T @ H_inv @ Z + 0.5 * Z.T @ H_inv @ v @ a_filtered_grad.T ) @ I_KZ + Z.T @ F_inv @ Z - Z.T @ F_inv @ v @ v.T @ F_inv @ Z

    return grad_P_hat

### Gradient with respect to **y**

From the article we have :

$$
\begin{align}
&\frac{dL}{dy_n} = K_n^T\frac{dL}{da_{n|n}} + \frac{dl_{n|n-1}}{dy_n} \quad &\text{(equation 24)} \\
&\frac{dl_{n|n-1}}{dy_n} = 2F^{-1}v_n \quad &\text{(equation 31)} \\
&\frac{dL}{da_{n|n}} = T_n^T \frac{dL}{da_{n+1|n}} \quad &\text{see gradient with respect to} \quad a_{n|n} \\
\end{align}
$$

Givent this two equations, we now have :
$$
\begin{align}
&\frac{dL}{dy_n} = K_n^TT_n^T\frac{dL}{da_{n+1|n}} + 2F^{-1}v_n
\end{align}
$$


In [72]:
def grad_y(inp, out, out_grad):
    y, a, P, T, Z, H, Q = inp
    a_hat_grad, P_h_grad, y_grad = out_grad

    y_hat = Z.dot(a)
    v = y - y_hat

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H
    F_inv = pt.linalg.inv(F)

    K = PZT.dot(F_inv) 
    
    return K.T @ T.T @ a_hat_grad + 2 * F_inv @ v

### Gradient with respect to Q

From the article we have :

$$
\begin{align}
\frac{dL}{dQ_n} = \frac{dL}{dP_{n|n-1}} & \quad \text{(equation 25)}
\end{align}
$$

In [73]:
def grad_Q(inp, out, out_grad):
    _, P_h_grad, _ = out_grad
    return P_h_grad

### Gradient with respect to **H**

From the article we have :

$$
\begin{align}
&\frac{dL}{dH_n} = K_n^T\frac{dL}{dP_{n|n}}K_n 
- \frac{1}{2} K_n^T \frac{dL}{da_{n|n}} v_n^T F^{-1}
- \frac{1}{2} S_n^{-1} v_n (\frac{dL}{da_{n|n}})^T K_n
+ \frac{dl_{n|n-1}}{dH_n} 
\quad &\text{(equation 26)} \\
&\frac{dl_{n|n-1}}{dH_n} = F^{-1} - F_n^{-1} v_n v_n^T F_n^{-1} 
\quad &\text{(equation 31)} \\
&\frac{dL}{da_{n|n}} = T_n^T \frac{dL}{da_{n+1|n}} \quad &\text{see gradient with respect to} \quad a_{n|n} \\
&\frac{dL}{dP_{n|n}} = T_n^T \frac{dL}{dP_{n+1|n}} T_n \quad &\text{see gradient with respect to} \quad P_{n|n}
\end{align}
$$

Givent this two equations, we now have :
$$
\begin{align}
&\frac{dL}{dH_n} = K_n^T T_n^T \frac{dL}{dP_{n+1|n}} T_n K_n 
- \frac{1}{2} K_n^T T_n^T \frac{dL}{da_{n+1|n}} v_n^T F^{-1}
- \frac{1}{2} F_n^{-1} v_n (T_n^T \frac{dL}{da_{n+1|n}})^T K_n
+ F^{-1} - F_n^{-1} v_n v_n^T F_n^{-1}
\end{align}
$$


In [74]:
def grad_H(inp, out, out_grad):
    y, a, P, T, Z, H, Q = inp
    a_hat_grad, P_h_grad, y_grad = out_grad
    
    y_hat = Z.dot(a)
    v = y - y_hat

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H
    F_inv = pt.linalg.inv(F)

    K = PZT.dot(F_inv)

    v = v.dimshuffle(0, 'x')
    a_hat_grad = a_hat_grad.dimshuffle(0, 'x') 

    a_filtered_grad = T.T @ a_hat_grad
    P_filtered_grad = T.T @ P_h_grad @ T

    return K.T @ P_filtered_grad @ K - 0.5 * K.T @ a_filtered_grad @ v.T @ F_inv - 0.5 *  F_inv @ v @ a_filtered_grad.T @ K + F_inv - F_inv @ v @ v.T @ F_inv

### Gradient with respect to **T**

This gradient was not given in the article. Here are the steps that got me to this expression :

1 - Only $x_{n|n-1}$ and $P_{n|n-1}$ depends on $T_n$. Hence :
$$
\frac{\partial L}{\partial T} = \frac{\partial L}{\partial x_{n|n-1}} \frac{\partial x_{n|n-1}}{\partial T} + \frac{\partial L}{\partial P_{n|n-1}} \frac{\partial T}{\partial P_{n|n-1}}
$$
2 - Using the equation (11) and (12) of the article, on the (1), we directly got that :
$$
\frac{\partial L}{\partial x_{n|n-1}} \frac{\partial x_{n|n-1}}{\partial T} = \frac{\partial L}{\partial x_{n|n-1}} x_{n-1|n-1}^T
$$
3 - Recognizing the first quadratic form in the equation (2), and using equation (11) we got :
$$
\frac{\partial L}{\partial P_{n|n-1}} \frac{\partial P_{n|n-1}}{\partial T^T} = P_{n|n-1}T_n^T \frac{\partial L}{\partial P_{n|n-1}}^T + P_{n|n-1}^T T_n^T \frac{\partial L}{\partial P_{n|n-1}}
$$
4 - Now transposing to get the dependencies on T :
$$
\frac{\partial L}{\partial P_{n|n-1}} \frac{\partial P_{n|n-1}}{\partial T} = \frac{\partial L}{\partial P_{n|n-1}} T_n P_{n|n-1}^T +\frac{\partial L}{\partial P_{n|n-1}}^T T_n P_{n|n-1}
$$
5 - Finally, we have :
$$
\frac{\partial L}{\partial T} = \frac{\partial L}{\partial x_{n|n-1}} x_{n-1|n-1}^T + \frac{\partial L}{\partial P_{n|n-1}} T_n P_{n|n-1}^T +\frac{\partial L}{\partial P_{n|n-1}}^T T_n P_{n|n-1}
$$

In [75]:
def grad_T(inp, out, out_grad):
    y, a, P, T, Z, H, Q = inp
    a_hat_grad, P_h_grad, y_grad = out_grad

    y_hat = Z.dot(a)
    v = y - y_hat

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H
    F_inv = pt.linalg.inv(F)

    K = PZT.dot(F_inv)
    I_KZ = pt.eye(a.shape[0]) - K.dot(Z)

    v = v.dimshuffle(0, 'x')
    a = a.dimshuffle(0, 'x')
    a_hat_grad = a_hat_grad.dimshuffle(0, 'x')

    a_filtered = a + K.dot(v)
    P_filtered = I_KZ @ P

    return a_hat_grad @ a_filtered.T + P_h_grad @ T @ P_filtered.T + P_h_grad.T @ T @ P_filtered

### Gradient with respect to Z

To obtain this gradient, I used the matrix differential + trace trick. So we consider that $Z_n$ influences the loss directly through $v_n$ and $F_n$, and indirectly through backpropagation through $P_{n|n}$ and $a_{n|n}$. Then $\frac{dL}{dZ_n}$ is the matrix that verify :

$$
dL = tr((\frac{dL}{dv_n})^T dv_n + (\frac{dL}{dF_n})^T dF_n + (\frac{dL}{dP_{n|n}})^T dP_{n|n} + (\frac{dL}{da_{n|n}})^T da_{n|n}) = tr((\frac{dL}{dZ_n})^TdZ_n)
$$

It is worth also writing that we'll use simple trace tricks :
- **transpose invariance** : $tr(A) = tr(A^T)$ ; 
- **ciclicity of the trace** : $tr(ABC) = tr(CAB) = tr(BCA)$. 

#### **First term** : $(\frac{dL}{dv_n})^T dv_n$

First noticing that :

$$
\begin{align}
\frac{dL}{dy_n} &= \frac{dL}{dv_n}\frac{dv_n}{dy_n} = \frac{dL}{dv_n} I = \frac{dL}{dv_n} \\
dv_n &= - dZ_n a_{n|n-1}
\end{align}
$$

And since we directly got from equation (31) of the paper that :

$$
\begin{align}
\frac{dL}{dy_n} = \frac{dl_{n|n-1}}{dy_n} = 2 F_n^{-1} v_n
\end{align}
$$

We ultimately have that :

$$
\begin{align}
(\frac{dL}{dv_n})^T dv_n &= ( 2 F_n^{-1} v_n )^T (- dZ_n a_{n|n-1}) = - 2 v_n^T F_n^{-T} dZ_n a_{n|n-1}
\end{align}
$$

Now, using trace tricks :

$$
\begin{align}
tr((\frac{dL}{dv_n})^T dv_n) = tr( - 2 F_n^{-1} v_n dZ_n a_{n|n-1} ) = tr( (- 2  F_n^{-1} v_n a_{n|n-1}^T)^T dZ_n)
\end{align}
$$

#### **Second term** : $(\frac{dL}{dF_n})^T dF_n$

Starting with differentiations and derivatives :

$$
\begin{align}
dF_n &= dZ_n P_{n|n-1} Z_n^T + Z_n P_{n|n-1} (dZ_n)^T \\
d(F_n^{-1}) &= -F_n^{-1} dF_n F_n^{-1} \\
\frac{dL}{dF_n} &= F_n^{-1} - F_n^{-1} z_n z_n^T F_n^{-1}\\
\end{align}
$$

Now, using the trace trick :

$$
\begin{align}
tr((\frac{dL}{dF_n})^T dF_n) &= tr((F_n^{-1} - F_n^{-1} z_n z_n^T F_n^{-1})^T (dZ_n P_{n|n-1} Z_n^T + Z_n P_{n|n-1} (dZ_n)^T)) \\
&=tr(F_n^{-T} dZ_n P_{n|n-1} Z_n^T + F_n^{-T} Z_n P_{n|n-1} (dZ_n)^T - F_n^{-T} z_n z_n^T F_n^{-T} dZ_n P_{n|n-1} Z_n^T - F_n^{-T} z_n z_n^T F_n^{-T} Z_n P_{n|n-1} (dZ_n)^T) \\
&=tr(P_{n|n-1} Z_n^T F_n^{-T} dZ_n + P_{n|n-1}^T Z_n^T F_n^{-1} dZ_n - P_{n|n-1} Z_n^T F_n^{-T} z_n z_n^T F_n^{-T} dZ_n - P_{n|n-1} F_n^{-1} z_n z_n^T F_n^{-1} dZ_n) \\
&=tr((F_n^{-1} Z_n P_{n|n-1}^T + F_n^{-T} Z_n P_{n|n-1} - F_n^{-1} z_n z_n^T F_n^{-1} Z_n P_{n|n-1}^T - F_n^{-T} z_n z_n^T F_n^{-T} Z_n P_{n|n-1}^T)^T dZ_n)
\end{align}
$$

Noticing that $P_{n|n-1}^T = P_{n|n-1}$ and $F_n^T = F_n$ :

$$
\begin{align}
tr((\frac{dL}{dF_n})^T dF_n) &= tr((2 F_n^{-1} Z_n P_{n|n-1} - 2 F_n^{-1} z_n z_n^T F_n^{-1} Z_n P_{n|n-1})^T dZ_n)
\end{align}
$$

#### **Third term** : $(\frac{dL}{dP_{n|n}})^T dP_{n|n}$

Starting with differencciations :

$$
\begin{align}
dP_{n|n} &= - (dK_n Z_n + K_n dZ_n) P_{n|n-1} \\
&= - (P_{n|n-1}((dZ_n)^T S_n^{-1} + Z_n^T d(S_n^{-1})) Z_n + K_n dZ_n) P_{n|n-1} \\
&= - (P_{n|n-1}((dZ_n)^T S_n^{-1} - Z_n^T S_n^{-1} dS_n S_n^{-1}) Z_n + K_n dZ_n) P_{n|n-1} \\
&= - (P_{n|n-1}((dZ_n)^T S_n^{-1} - Z_n^T S_n^{-1} (dZ_n P_{n|n-1} Z_n^T + Z_n P_{n|n-1} (dZ_n)^T) S_n^{-1}) Z_n + K_n dZ_n) P_{n|n-1} \\
&= - P_{n|n-1}(dZ_n)^T S_n^{-1} Z_n P_{n|n-1} \\
&+ P_{n|n-1} Z_n^T S_n^{-1} dZ_n P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1} \\
&+ P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1} (dZ_n)^T S_n^{-1} Z_n P_{n|n-1} \\
&- K_n dZ_n P_{n|n-1} \\
\end{align}
$$

Term by term, let's use the trace trick to get $dZ_n$ at the right place :

$$
\begin{align}
&tr((\frac{dL}{dP_{n|n}})^T P_{n|n-1}(dZ_n)^T S_n^{-1} Z_n P_{n|n-1}) = tr(P_{n|n-1}^T Z_n^T S_n^{-T} dZ_n P_{n|n-1}^T \frac{dL}{dP_{n|n}}) = tr(P_{n|n-1}^T \frac{dL}{dP_{n|n}} P_{n|n-1}^T Z_n^T S_n^{-T} dZ_n) \\
&tr((\frac{dL}{dP_{n|n}})^T P_{n|n-1} Z_n^T S_n^{-1} dZ_n P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1}) = tr(P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1} (\frac{dL}{dP_{n|n}})^T P_{n|n-1} Z_n^T S_n^{-1} dZ_n) \\
&tr((\frac{dL}{dP_{n|n}})^T P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1} (dZ_n)^T S_n^{-1} Z_n P_{n|n-1}) = tr(P_{n|n-1}^T Z_n^T S_n^{-T} dZ_n P_n{n|n-1}^T Z_n^T S_n^{-T} Z_n P_{n|n-1}^T \frac{dL}{dP_{n|n}}) \\
&= tr(P_{n|n-1}^T Z_n^T S_n^{-T} Z_n P_{n|n-1}^T \frac{dL}{dP_{n|n}} P_{n|n-1}^T Z_n^T S_n^{-T} dZ_n) \\
&tr((\frac{dL}{dP_{n|n}})^T K_n dZ_n P_{n|n-1}) = tr(P_{n|n-1} (\frac{dL}{dP_{n|n}})^T K_n dZ_n) \\
\\
\end{align}
$$

So the contibution of $Z_n$ through $P_{n|n}$ is :

$$
\begin{align}
&tr((\frac{dL}{dP_{n|n}})^T dP_{n|n}) = tr((- S_n^{-1} Z_n P_{n|n-1} (\frac{dL}{dP_{n|n}})^T P_{n|n-1} + K_n^T \frac{dL}{dP_{n|n}} P_{n|n-1}^T  Z_n^T K_n^T + S_n^{-1} Z_n P_{n|n-1} (\frac{dL}{dP_{n|n}})^T K_n Z_n P_{n|n-1} - K_n^T \frac{dL}{dP_{n|n}} P_{n|n-1}^T)^T K_n)^T dZ_n)
\end{align}
$$

#### **Fourth term** : $(\frac{dL}{da_{n|n}})^T da_{n|n}$

Starting with differenciations :

$$
\begin{align}
da_{n|n} &= dK_n v_n + K_n dv_n \\
&= P_{n|n-1}((dZ_n)^T S_n^{-1} + Z_n^T d(S_n^{-1})) v_n - K_n dZ_n a_{n|n-1} \\
&= P_{n|n-1}((dZ_n)^T S_n^{-1} - Z_n^T S_n^{-1} dS_n S_n^{-1}) v_n - K_n dZ_n a_{n|n-1} \\
&= P_{n|n-1}((dZ_n)^T S_n^{-1} - Z_n^T S_n^{-1} (dZ_n P_{n|n-1} Z_n^T + Z_n P_{n|n-1} (dZ_n)^T) S_n^{-1}) v_n - K_n dZ_n a_{n|n-1} \\
&= P_{n|n-1} (dZ_n)^T S_n^{-1} v_n \\
&- P_{n|n-1} Z_n^T S_n^{-1} dZ_n P_{n|n-1} Z_n^T S_n^{-1} v_n \\
&- P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1} (dZ_n)^T S_n^{-1} v_n \\
&- K_n dZ_n a_{n|n-1} \\
\end{align}
$$

Term by term, let's use the trace trick to get $dZ_n$ at the right place :

$$
\begin{align}
&tr((\frac{dL}{da_{n|n}})^T P_{n|n-1} (dZ_n)^T S_n^{-1} v_n) = tr(v_n^T S_n^{-T} dZ_n P_{n|n-1}^T \frac{dL}{da_{n|n}}) = tr(P_{n|n-1}^T \frac{dL}{da_{n|n}} v_n^T S_n^{-T} dZ_n) \\
&tr((\frac{dL}{da_{n|n}})^T P_{n|n-1} Z_n^T S_n^{-1} dZ_n P_{n|n-1} Z_n^T S_n^{-1} v_n) = tr(P_{n|n-1} Z_n^T S_n^{-1} v_n (\frac{dL}{da_{n|n}})^T P_{n|n-1} Z_n^T S_n^{-1} dZ_n) \\
&tr((\frac{dL}{da_{n|n}})^T P_{n|n-1} Z_n^T S_n^{-1} Z_n P_{n|n-1} (dZ_n)^T S_n^{-1} v_n) = tr(v_n^T S_n^{-T} dZ_n P_n{n|n-1}^T Z_n^T S_n^{-T} Z_n P_{n|n-1}^T \frac{dL}{da_{n|n}}) \\
&= tr(P_n{n|n-1}^T Z_n^T S_n^{-T} Z_n P_{n|n-1}^T \frac{dL}{a_{n|n}} v_n^T S_n^{-T} dZ_n) \\
&tr((\frac{dL}{da_{n|n}})^T K_n dZ_n a_{n|n-1}) = tr(a_{n|n-1} (\frac{dL}{dda_{n|n}})^T K_n dZ_n) \\
\\
\end{align}
$$

So the contibution of $Z_n$ through $a_{n|n}$ is :
$$
\begin{align}
&tr((\frac{dL}{da_{n|n}})^T da_{n|n}) = tr((S_n^{-1} v_n (\frac{dL}{da_{n|n}})^T P_{n|n-1} - K_n^T \frac{dL}{da_{n|n}} v_n^T K_n^T - S_n^{-1} v_n (\frac{dL}{da_{n|n}})^T K_n Z_n P_{n|n-1} - K_n^T \frac{dL}{da_{n|n}} a_{n|n-1}^T)^T dZ_n) \\
\end{align}
$$

#### **To sum up** :

In the end, the gradient of $L$ with respect to $Z_n$ is :

$$
\begin{align}
\frac{dL}{dZ_n} &= - 2 F_n^{-1} v_n a_{n|n-1}^T\\
&+ 2 F_n^{-1} Z_n P_{n|n-1} - 2 F_n^{-1} z_n z_n^T F_n^{-1} Z_n P_{n|n-1}\\
&+ S_n^{-1} v_n (\frac{dL}{da_{n|n}})^T P_{n|n-1} - K_n^T \frac{dL}{da_{n|n}} v_n^T K_n^T - S_n^{-1} v_n (\frac{dL}{da_{n|n}})^T K_n Z_n P_{n|n-1} - K_n^T \frac{dL}{da_{n|n}} a_{n|n-1}^T \\
&- S_n^{-1} Z_n P_{n|n-1} (\frac{dL}{dP_{n|n}})^T P_{n|n-1} + K_n^T \frac{dL}{dP_{n|n}} P_{n|n-1}^T  Z_n^T K_n^T + S_n^{-1} Z_n P_{n|n-1} (\frac{dL}{dP_{n|n}})^T K_n Z_n P_{n|n-1} - K_n^T \frac{dL}{dP_{n|n}} P_{n|n-1}^T \\
\end{align}
$$

In [76]:
def grad_Z(inp, out, out_grad):

    y, a, P, T, Z, H, Q = inp
    a_h_grad, P_h_grad, y_grad = out_grad

    y_hat = Z.dot(a)
    v = y - y_hat

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H
    F_inv = pt.linalg.inv(F)
    K = PZT.dot(F_inv)

    v = v.dimshuffle(0, 'x')
    a = a.dimshuffle(0, 'x')
    a_h_grad = a_h_grad.dimshuffle(0, 'x')

    a_filtered_grad = T.T @ a_h_grad
    P_filtered_grad = T.T @ P_h_grad @ T

    # Contribution via Pnn

    term_P_1 = - F_inv @ Z @ P @ P_filtered_grad.T @ P
    term_P_2 = K.T @ P_filtered_grad @ P.T @ Z.T @ K.T
    term_P_3 = F_inv @ Z @ P @ P_filtered_grad.T @ K @ Z @ P
    term_P_4 = - K.T @ P_filtered_grad @ P.T

    contrib_P = term_P_1 + term_P_2 + term_P_3 + term_P_4

    # Contibution via xnn

    term_x_1 = F_inv @ v @ a_filtered_grad.T @ P
    term_x_2 = - K.T @ a_filtered_grad @ v.T @ K.T
    term_x_3 = - F_inv @ v @ a_filtered_grad.T @ K @ Z @ P
    term_x_4 = - K.T @ a_filtered_grad @ a.T

    contrib_x = term_x_1 + term_x_2 + term_x_3 + term_x_4

    # Contribution via Fn
    
    term_F_1 = 2 * F_inv @ Z @ P 
    term_F_2 = - 2 *F_inv @ v @ v.T @ F_inv @ Z @ P

    contrib_F = term_F_1 + term_F_2

    # Contribution via vn
    contrib_v = - 2 * F_inv @ v @ a.T
    
    return  contrib_x + contrib_P + contrib_F + contrib_v

### Total grad

In [77]:
def custom_grad(inp, out, out_grad):
    y, a, P, T, Z, H, Q = inp
    a_filtered, P_filtered, y_hat = out
    a_hat_grad, P_hat_grad, y_grad = out_grad

    PZT = P.dot(Z.T)
    F = Z.dot(PZT) + H

    y_hat = Z.dot(a)
    v = y - y_hat

    H_inv = pt.linalg.inv(H)
    F_inv = pt.linalg.inv(F)

    K = PZT.dot(F_inv)
    I_KZ = pt.eye(a.shape[0]) - K.dot(Z)
    
    grad_a_pred = I_KZ.T @ T.T @ a_hat_grad - 2 * Z.T @ F_inv @ v
    grad_y = K.T @ T.T @ a_hat_grad + 2 * F_inv @ v


    a_hat_grad = a_hat_grad.dimshuffle(0, 'x')
    v = v.dimshuffle(0, 'x')
    
    P_filtered_grad = T.T @ P_hat_grad @ T
    a_filtered_grad = T.T @ a_hat_grad  

    grad_P_hat = I_KZ.T @ ( P_filtered_grad + 0.5 * a_filtered_grad @ v.T @ H_inv @ Z + 0.5 * Z.T @ H_inv @ v @ a_filtered_grad.T ) @ I_KZ + Z.T @ F_inv @ Z - Z.T @ F_inv @ v @ v.T @ F_inv @ Z
    grad_Z = None
    grad_T = None
    grad_Q = P_hat_grad
    grad_H = K.T @ P_filtered_grad @ K - 0.5 * K.T @ a_filtered_grad @ v.T @ F_inv - 0.5 * F_inv @ v @ a_filtered_grad.T @ K + F_inv - F_inv @ v @ v.T @ F_inv

    return [grad_P_hat,
            grad_a_pred,
            grad_y,
            grad_Z,
            grad_T,
            grad_Q,
            grad_H]


## Custom Kalman Filter

In [78]:
y_sym = pt.vector("y")

kalman_step_op = OpFromGraph(
    inputs=[y_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
    outputs=kalman_step(y_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym),
    lop_overrides=[grad_y, grad_a_hat, grad_P_hat, grad_T, grad_Z, grad_H, grad_Q],
    inline=True
)

outputs_info = [None, a0_sym, None, None, P0_sym, None, None]

results_op, updates = pytensor.scan(
    kalman_step_op,
    sequences=[data_sym],
    outputs_info=outputs_info,
    non_sequences=[T_sym, Z_sym, H_sym, Q_sym],
    strict=False,
)
# --- Loss ---
a_upd_op, a_pred_op, y_hat_op, P_upd_op, P_pred_op, obs_cov, ll_op = results_op
loss_op = pt.sum(ll_op)

## Handmade Numpy Backpropagation 

In [79]:
def compute_grad_a0(observations, a0, P0, a_pred_seq, P_pred_seq, Z, H, T):
    # Constant
    SHAPE_a0 = a0.shape[0]
    NB_obs = len(observations)

    # Initialisation for the backprop
    PZT = P_pred_seq[-2].dot(Z.T)
    F = Z.dot(PZT) + H
    F_inv = np.linalg.solve(F, np.eye(F.shape[0]))
    
    grad = [0 for _ in range(NB_obs)]
    grad[-1] = - 2 * Z.T @ F_inv @ (observations[-1] - Z @ a_pred_seq[-2])

    # Backprop
    for i in range(3, NB_obs+1):

        PZT = P_pred_seq[-i].dot(Z.T)
        F = Z.dot(PZT) + H
        F_inv = np.linalg.solve(F, np.eye(F.shape[0]))

        K = PZT.dot(F_inv)
        I_KZ = np.eye(SHAPE_a0) - K.dot(Z)

        grad[1-i] = I_KZ.T @ T.T @ grad[2-i] - (2 * Z.T @ F_inv @ (observations[1-i] - Z @ a_pred_seq[-i])).T   

    # Last iter with a0/P0
    PZT = P0.dot(Z.T)
    F = Z.dot(PZT) + H
    F_inv = np.linalg.solve(F, np.eye(F.shape[0]))

    K = PZT.dot(F_inv)
    I_KZ = np.eye(SHAPE_a0) - K.dot(Z)

    grad[0] = I_KZ.T @ T.T @ grad[1] - (2 * Z.T @ F_inv @ (observations[0] - Z @ a0)).T

    return grad

## Speed observation

### Benchmark for pytensor computed gradients

In [80]:
def benchmark_kalman_gradients(loss, obs_data, a0, P0, T, Z, R, H, Q):
    results = defaultdict(dict)
    exec_time = 0

    grad_list = pt.grad(loss, [data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym])
    f_grad = pytensor.function(
        inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
        outputs=grad_list,
    )

    for _ in range(20):
    
        # --- exécution ---
        t0 = perf_counter()
        _ = f_grad(
            obs_data[:, np.newaxis],
            a0,
            P0,
            T,
            Z,
            H,
            R @ Q @ R.T,
        )
        t1 = perf_counter()
        exec_time += (t1 - t0)/20
    
    
    results["exec_time"] = exec_time

    return results

### Benchmark for numpy computed gradient

In [81]:
def benchmark_kalman_gradients_np(a_pred_seq, P_pred_seq, obs_data, a0, P0, T, Z, R, H, Q):
        results = defaultdict(dict)
        forward_pass = 0
        backprop = 0
        kalman_fn = pytensor.function(inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
                                      outputs=(a_pred_seq, P_pred_seq))

        for _ in range(20):

                # --- forward pass ---
                t0 = perf_counter()
                a_pred, P_pred = kalman_fn(obs_data[:, np.newaxis],
                                        a0,
                                        P0,
                                        T,
                                        Z,
                                        H,
                                        R@Q@R.T,)
                t1 = perf_counter()
                forward_pass += (t1 - t0)/20
                

                # --- Backprop ---
                t0 = perf_counter()
                _ = compute_grad_a0(
                        obs_data,
                        a0,
                        P0,
                        a_pred,
                        P_pred,
                        Z,
                        H,
                        T,)
                t1 = perf_counter()
                backprop += (t1 - t0)/20

        results["Forward pass"] = forward_pass      
        results["Backprop"] = backprop

        return results

### Comparison

In [82]:
%%prun
results = benchmark_kalman_gradients(loss, obs_data, a0, P0, T, Z, R, H, Q)

c:\Users\jeanv\miniconda3\envs\CausalPy\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:954: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


         16655700 function calls (16473469 primitive calls) in 19.496 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
137658/137401    0.869    0.000    0.941    0.000 utils.py:229(__eq__)
    37352    0.852    0.000    2.619    0.000 elemwise.py:662(perform)
    22220    0.644    0.000    0.676    0.000 blas.py:894(perform)
   248821    0.610    0.000    8.445    0.000 op.py:536(rval)
       40    0.603    0.015    8.766    0.219 op.py:1663(perform)
    79609    0.577    0.000    0.577    0.000 {method 'reduce' of 'numpy.ufunc' objects}
1286912/1286852    0.457    0.000    0.863    0.000 {built-in method builtins.isinstance}
    37352    0.395    0.000    0.771    0.000 elemwise.py:730(_check_runtime_broadcast)
    52632    0.380    0.000    1.348    0.000 _util.py:491(_asarray_validated)
  4060/20    0.379    0.000    9.575    0.479 vm.py:380(__call__)
    12120    0.350    0.000    0.373    0.000 blas.py:182(perform)
   

In [83]:
print(results)

defaultdict(<class 'dict'>, {'exec_time': 0.47910417499952024})


In [84]:
results_op = benchmark_kalman_gradients(loss_op, obs_data, a0, P0, T, Z, R, H, Q)

c:\Users\jeanv\miniconda3\envs\CausalPy\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:954: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


In [85]:
print(results_op)

defaultdict(<class 'dict'>, {'exec_time': 0.585526670049876})


In [86]:
results_np = benchmark_kalman_gradients_np(a_pred_seq, P_pred_seq, obs_data, a0, P0, T, Z, R, H, Q)

In [87]:
print(results_np)

defaultdict(<class 'dict'>, {'Forward pass': 0.05328435508999973, 'Backprop': 0.013503129919990897})


## Error observation

### Comparing the gradient with respect to a0

In [88]:
# First the classic way with autodiff

grad_list = pt.grad(loss, [a0_sym])
f_grad = pytensor.function(
    inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
    outputs=grad_list,
)

grad_a0 = f_grad(obs_data[:, np.newaxis], a0, P0, T, Z, H, R @ Q @ R.T)

# Now using our OpFromGraph custom gradient

grad_list_op = pt.grad(loss_op, [a0_sym])
f_grad = pytensor.function(
    inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
    outputs=grad_list_op,
)

grad_a0_op = f_grad(obs_data[:, np.newaxis], a0, P0, T, Z, H, R @ Q @ R.T)

# And here using our handmaid numpy backprop

kalman_fn = pytensor.function(inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
                              outputs=(a_pred_seq, P_pred_seq))
a_pred, P_pred = kalman_fn(obs_data[:, np.newaxis], a0, P0, T, Z, H, R@Q@R.T)

grad_a0_np = compute_grad_a0(obs_data, a0, P0, a_pred, P_pred, Z, H, T)[0]

c:\Users\jeanv\miniconda3\envs\CausalPy\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:954: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
c:\Users\jeanv\miniconda3\envs\CausalPy\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:954: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


In [89]:
print("Comparison between classic a0 gradient and our custom OpFromGraph :", np.allclose(grad_a0,  grad_a0_op))
print("Comparison between classic a0 gradient and our handmade NumPy backprop :", np.allclose(grad_a0,  grad_a0_np))

Comparison between classic a0 gradient and our custom OpFromGraph : True
Comparison between classic a0 gradient and our handmade NumPy backprop : True


In [90]:
# First the classic way with autodiff

grad_list = pt.grad(loss, [data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym])
f_grad = pytensor.function(
    inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
    outputs=grad_list,
)

grad_a0 = f_grad(obs_data[:, np.newaxis], a0, P0, T, Z, H, R @ Q @ R.T)

# Now using our OpFromGraph custom gradient

grad_list_op = pt.grad(loss_op, [data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym])
f_grad = pytensor.function(
    inputs=[data_sym, a0_sym, P0_sym, T_sym, Z_sym, H_sym, Q_sym],
    outputs=grad_list_op,
)

grad_a0_op = f_grad(obs_data[:, np.newaxis], a0, P0, T, Z, H, R @ Q @ R.T)

c:\Users\jeanv\miniconda3\envs\CausalPy\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:954: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
c:\Users\jeanv\miniconda3\envs\CausalPy\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:954: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


In [91]:
print("Comparison between classic y gradient and our custom OpFromGraph :", np.allclose(grad_a0[0],  grad_a0_op[0]))
print("Comparison between classic a0 gradient and our custom OpFromGraph :", np.allclose(grad_a0[1],  grad_a0_op[1]))
print("Comparison between classic P0 gradient and our custom OpFromGraph :", np.allclose((grad_a0[2] + grad_a0[2].T)/2,  grad_a0_op[2]))
print("Comparison between classic T gradient and our custom OpFromGraph :", np.allclose(grad_a0[3],  grad_a0_op[3]))
print("Comparison between classic Z gradient and our custom OpFromGraph :", np.allclose(grad_a0[4],  grad_a0_op[4]))
print("Comparison between classic H gradient and our custom OpFromGraph :", np.allclose(grad_a0[5],  grad_a0_op[5]))
print("Comparison between classic Q gradient and our custom OpFromGraph :", np.allclose((grad_a0[6] + grad_a0[6].T)/2,  grad_a0_op[6]))

Comparison between classic y gradient and our custom OpFromGraph : True
Comparison between classic a0 gradient and our custom OpFromGraph : True
Comparison between classic P0 gradient and our custom OpFromGraph : True
Comparison between classic T gradient and our custom OpFromGraph : True
Comparison between classic Z gradient and our custom OpFromGraph : True
Comparison between classic H gradient and our custom OpFromGraph : True
Comparison between classic Q gradient and our custom OpFromGraph : True
